Lab 2 

Seth Abuhamdeh

34937889

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import math as math
from pandas.api.types import is_bool_dtype

Preparing the Data

In [ ]:
df = pd.read_csv("card_fraud.csv")
cols = ['distance_from_home', 'distance_from_last_transaction','ratio_to_median_purchase_price','repeat_retailer', 'used_chip','used_pin_number','online_order']
X_train, X_test, y_train , y_test = train_test_split(df[cols], df["fraud"], train_size = .1, random_state = 18)
df

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,True,True,False,False,Not Fraud
1,10.829943,0.175592,1.294219,True,False,False,False,Not Fraud
2,5.091079,0.805153,0.427715,True,False,False,True,Not Fraud
3,2.247564,5.600044,0.362663,True,True,False,True,Not Fraud
4,44.190936,0.566486,2.222767,True,True,False,True,Not Fraud
...,...,...,...,...,...,...,...,...
999995,2.207101,0.112651,1.626798,True,True,False,False,Not Fraud
999996,19.872726,2.683904,2.778303,True,True,False,False,Not Fraud
999997,2.914857,1.472687,0.218075,True,True,False,True,Not Fraud
999998,4.258729,0.242023,0.475822,True,False,False,True,Not Fraud


In [ ]:
DeepnoteChart(df, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#2266D3","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"fraud","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"count","formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{},"usermeta":{"tooltipDefaultMode":true}}""")

If we always predict not fraud, we would have around a 91% accuracy score for both the train set and test set. There wouldn't be any overfitting because we are putting all the weight on a single variable, we are actually underfitting instead. 

Logistic Regression

In [ ]:
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)
LogReg.score(X_train,y_train)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9593

In [ ]:
LogReg.score(X_test,y_test)

0.9579166666666666

accuracy is close to 96%, we are not very concerned with overfitting since 91% of the entries are already fraud, so our model having only a 5% increase is more likely caused by the model fitting the data well rather than just overfitting.

In [ ]:
y_pred = LogReg.predict(X_test)
y_pred
confusion_matrix(y_test,y_pred)

array([[ 46400,  32371],
       [  5504, 815725]])

The diaganol entries since this would be the amount of data pts predicted correctly by the model.  The bank would like to see what factors the model considers the most to predict whether a transaction is fraudulent.

### Naive Bayes

In [ ]:
df_train = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
df_train

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,5.269072,3.895352,0.459102,True,False,False,True,Not Fraud
1,10.222046,0.171404,1.296534,True,False,False,False,Not Fraud
2,5.918397,0.116066,2.167989,True,False,False,True,Not Fraud
3,0.979114,4.798067,6.826781,False,True,False,False,Not Fraud
4,195.568268,0.534504,0.630321,True,False,False,True,Fraud
...,...,...,...,...,...,...,...,...
99995,12.213984,0.406614,0.777585,True,False,False,False,Not Fraud
99996,0.410121,26.478868,0.821981,False,False,False,True,Not Fraud
99997,2.783387,2.612634,1.939725,True,False,False,True,Not Fraud
99998,13.547140,0.379531,5.104000,True,True,False,True,Fraud


In [ ]:
prior_dct = {
    "Fraud" : .917,
    "Not Fraud" : .083
}


In [ ]:
def Gaussian_helper (df,k,col):
    output_dct ={}
    filtered_df = df[df["fraud"]==k]
    output_dct["mean"] = filtered_df[col].mean()
    output_dct["std"] = filtered_df[col].std()
    return output_dct

def Boolean_helper (df,k,col):
    
    filtered_df = df[df["fraud"]==k]
    true_count = filtered_df[col].sum()
    false_count = filtered_df[col].count() - true_count

    total = len(filtered_df)
    true_prop = true_count/ total
    false_prop = false_count / total
    output = {True: true_prop, False: false_prop}
    return output





In [ ]:
mean, std = Gaussian_helper (df, "Fraud", "distance_from_home")
mean


'mean'

In [ ]:
Boolean_helper(df,"Not Fraud","used_chip")

{True: 0.35940179509685, False: 0.64059820490315}

In [ ]:

df_train.groupby("fraud").mean()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order
fraud,,,,,,,
Fraud,67.586212,12.830475,6.041095,0.883804,0.259036,0.003128,0.947405
Not Fraud,22.779186,4.279180,1.427654,0.882979,0.359130,0.110006,0.622154


In [ ]:
from scipy.stats import norm

norm.pdf(X_train["distance_from_last_transaction"], loc=10, scale=4)

array([0.03112269, 0.00487311, 0.00470979, ..., 0.01812171, 0.00553022,
       0.02834549])

In [ ]:
temp_dct = {True: 0.71, False: 0.29}

X_train["used_chip"].map(temp_dct)

629362    0.29
793751    0.29
849158    0.29
394906    0.71
223707    0.29
          ... 
788158    0.29
35653     0.29
353400    0.29
356627    0.71
978218    0.29
Name: used_chip, Length: 100000, dtype: float64

In [ ]:
df_train.columns

Index(['distance_from_home', 'distance_from_last_transaction',
       'ratio_to_median_purchase_price', 'repeat_retailer', 'used_chip',
       'used_pin_number', 'online_order', 'fraud'],
      dtype='object')

In [ ]:


num_dct = {}
for k in ["Fraud", "Not Fraud"]:
    X_temp = X_train.copy()
    for col in X_temp.columns:
        if is_bool_dtype(X_temp[col]):
            props = Boolean_helper(df_train, k, col)
            X_temp[col] = X_temp[col].apply(lambda x : props[x])
        else:
            stats = Gaussian_helper(df_train, k, col)
            mean = stats["mean"]
            std_dev = stats["std"]
            X_temp[col] = X_temp[col].apply(lambda x : (1/(std_dev * ( 2 * math.pi ) ** 0.5))* math.exp(-0.5 * ((x-mean)/std_dev)**2))
    prior_class = df_train["fraud"].eq(k).mean()
    numerator = X_temp.apply(lambda row: row.prod()*prior_class, axis = 1)
    num_dct[k] = numerator
num_dct

{'Fraud': 629362    4.627054e-08
 793751    2.902501e-09
 849158    5.748785e-08
 394906    1.810504e-10
 223707    3.452811e-08
               ...     
 788158    2.712247e-09
 35653     6.201442e-09
 353400    5.611497e-08
 356627    2.517925e-08
 978218    4.118167e-09
 Length: 100000, dtype: float64,
 'Not Fraud': 629362    5.979386e-06
 793751    4.060340e-06
 849158    6.163951e-06
 394906    1.217667e-08
 223707    2.771365e-08
               ...     
 788158    3.924977e-06
 35653     5.134512e-07
 353400    6.311114e-06
 356627    8.765711e-07
 978218    1.466494e-07
 Length: 100000, dtype: float64}

In [ ]:
df_num = pd.DataFrame(num_dct)
df_num


,Fraud,Not Fraud
629362,4.627054e-08,5.979386e-06
793751,2.902501e-09,4.060340e-06
849158,5.748785e-08,6.163951e-06
394906,1.810504e-10,1.217667e-08
223707,3.452811e-08,2.771365e-08
...,...,...
788158,2.712247e-09,3.924977e-06
35653,6.201442e-09,5.134512e-07
353400,5.611497e-08,6.311114e-06
356627,2.517925e-08,8.765711e-07


In [ ]:

prob_fraud = df_num["Fraud"]
prob_not = df_num["Not Fraud"]


predicted_class = prob_fraud > prob_not

predicted_class = predicted_class.reset_index(drop=True)

actual_class = (df_train["fraud"] == 'Fraud')


correct_prop = (predicted_class == actual_class).mean()

correct_prop


0.9261

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1a3d6e0c-69e9-4386-98e0-a4f1d89bc209' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>